In [1]:
import os
os.chdir("/home/yangkai/codegen-detection/")
os.environ['CUDA_VISIBLE_DEVICES'] = '5,7'

In [3]:
from gltr_backend import LM

/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so'), PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


[2023-12-07 20:28:55,035] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
model_name = "../llm_checkpoints/CodeLlama-13b-Python-hf/"
n_gpus = 2
max_mem = '14GiB'
backend_model = LM(model_name_or_path=model_name,num_gpus=n_gpus,max_mem=max_mem,use_int8=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:39<00:00, 13.04s/it]

Loaded ../llm_checkpoints/CodeLlama-13b-Python-hf/ model!


In [5]:
# gen_model: gpt35
# rewrite_model: codellama
import json
from response_extraction import step1_extract_code,step2_code_filter
def json_load(fname):
    responses = [json.loads(line.strip()) for line in open(fname).readlines()]
    extracted = []
    for i in range(8):
        codes = []
        for resp in responses:
            try:
                code = step2_code_filter(step1_extract_code(resp['choices'][i]['message']['content']))
            except:
                code = ""
            codes.append(code)
        extracted.append(codes)
    return extracted

def load(fname):
    return [json.loads(line.strip()) for line in open(fname).readlines()]

dset = "apps"
rewrite_model = "codellama"
gen_model = "_codellama"
hum_pth = f"dataset/{dset}_hum_test.pkl"
gen_pth = f"dataset/{dset}_gen_test{gen_model}.pkl"

hum_rewrites = [load(f"prompt_gpt/{dset}_hum_test_{rewrite_model}_rewrite_resp{id}.jsonl") for id in range(1,9)]
gen_rewrites = [load(f"prompt_gpt/{dset}_gen_test{gen_model}_{rewrite_model}_rewrite_resp{id}.jsonl") for id in range(1,9)]
# gen_rewrites = json_load(f"{dset}_gen_test{gen_model}_{rewrite_model}_rewrite_resp21.jsonl")

In [6]:
import pandas as pd
import json

def data_prepare(orig_pth,rewrites):
    df = pd.read_pickle(orig_pth)[['extracted_full_func']]
    for i,rewrite in enumerate(rewrites):
        df[f'rewrite_{i}'] = rewrite
    
    return df

hum_df = data_prepare(hum_pth,hum_rewrites)
gen_df = data_prepare(gen_pth,gen_rewrites)

In [7]:
import numpy as np
import torch

def word_prob_gather_fn(word_prob):
    try:
        assert len(word_prob) > 0
    except:
        return 0
    
    log_prob = np.array(word_prob)
    avg_log_prob = log_prob.sum() / (len(log_prob) + 1e-6)
    # print(avg_log_prob)
    return avg_log_prob

def log_rank_gather_fn(word_rank):
    try:
        assert len(word_rank) > 0
    except:
        return 0
    log_rank = np.log(np.array(word_rank) + 1)
    avg_log_rank = log_rank.sum() / (len(log_rank) + 1e-6)
    return -avg_log_rank

def rank_gather_fn(word_rank):
    try:
        assert len(word_rank) > 0
    except:
        return 0
    log_rank = np.array(word_rank) + 1
    avg_log_rank = log_rank.sum() / (len(log_rank) + 1e-6)
    return -avg_log_rank

def entropy_gather_fn(entropy):
    try:
        assert len(entropy) > 0
    except:
        return 0
    entropy = np.array(entropy)
    
    avg_entropy = entropy.sum() / (len(entropy) + 1e-6)
    return -avg_entropy

@torch.inference_mode()
def get_score(text):
    select_ranks, select_probs, entropy = \
            backend_model.check_probabilities([text])
    
    log_prob = word_prob_gather_fn(select_probs)
    log_rank = log_rank_gather_fn(select_ranks)
    rank = rank_gather_fn(select_ranks)
    entropy = entropy_gather_fn(entropy)

    return log_prob,log_rank,rank,entropy


In [8]:
from tqdm import tqdm

def mean(x):
    return sum(x) / len(x)

def get_feat(df):
    bar = tqdm(total=len(hum_df))
    orig_feat = []
    re_feat = []
    for _,d in hum_df.iterrows():
        orig_func = d['extracted_full_func']
        log_prob,log_rank,rank,entropy = get_score(orig_func)
        orig_feat.append({'log_prob':log_prob,'log_rank':log_rank,'entropy':entropy})

        probs,log_ranks,entropys = [],[],[]
        for i in range(8):
            rewrite_func = d[f'rewrite_{i}']
            re_log_prob,re_log_rank,_,re_entropy = get_score(rewrite_func)
            probs.append(re_log_prob)
            log_ranks.append(re_log_rank)
            entropys.append(re_entropy)
        
        re_log_prob = mean(probs)
        re_log_rank = mean(log_ranks)
        re_entropy = mean(entropys)

        re_feat.append({'log_prob':re_log_prob,'log_rank':re_log_rank,'entropy':re_entropy})
        bar.update()
    df['log_prob'] = [item['log_prob'] for item in orig_feat]
    df['log_rank'] = [item['log_rank'] for item in orig_feat]
    df['entropy'] = [item['entropy'] for item in orig_feat]

    df['re_log_prob'] = [item['log_prob'] for item in re_feat]
    df['re_log_rank'] = [item['log_rank'] for item in re_feat]
    df['re_entropy'] = [item['entropy'] for item in re_feat]

    return df

## Load Results

In [1]:
import os
os.chdir("/home/yangkai/codegen-detection/")

In [7]:
import pandas as pd
gen_df = pd.read_pickle("ablation_out/codellama_rewrite_apps100_gpt4_token_score_gen.pkl")
hum_df = pd.read_pickle("ablation_out/codellama_rewrite_apps100_token_score_hum.pkl")

In [6]:
from sklearn.metrics import roc_auc_score
for metric in ['log_prob','log_rank','rank','entropy']:
    gen_score = (gen_df[f'{metric}'] - gen_df[f're_{metric}']).to_list()
    hum_score = (hum_df[f'{metric}'] - hum_df[f're_{metric}']).to_list()
    scores = gen_score + hum_score
    labels = [1] * len(gen_score) + [0] * len(hum_score)
    auc = roc_auc_score(labels,scores)
    print(metric,auc)


log_prob 0.5729465339854951
log_rank 0.7528740091077754
rank 0.6232838589981446
entropy 0.6038994771462304


0.6820492494518469

In [16]:
gen_df

,extracted_full_func,rewrite_0,rewrite_1,rewrite_2,rewrite_3,rewrite_4,rewrite_5,rewrite_6,rewrite_7,log_prob,log_rank,entropy,re_log_prob,re_log_rank,re_entropy
630,"import re\ndef starts_with_substring(string, s...","def starts_with_substring(string, substring):\...","import re\ndef starts_with_substring(string, s...","def starts_with_substring(string, substring):\...","def starts_with_substring(string, substring):\...","def starts_with_substring(string, substring):\...","import re\ndef starts_with_substring(string, s...","def starts_with_substring(string, substring):\...","import re\ndef starts_with_substring(string, s...",26.557527,-34.058154,-59.214699,16.922280,-29.263127,-44.985065
917,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,def find_min_subsets_with_distinct_elements(nu...,from collections import Counter\ndef find_min_...,def find_min_subsets_with_distinct_elements(nu...,44.431019,-37.522821,-75.259301,35.074903,-37.521411,-72.383630
626,def find_demlo_number(n):\n if n == 1:\n ...,def find_demlo_number(n):\n return n ** n,def find_demlo_number(n):\n demlo_num = 1\n...,def find_demlo_number(n):\n if n == 1:\n ...,def find_demlo_number(n):\n if n == 1:\n ...,def find_demlo_number(n):\n result = 1\n ...,def find_demlo_number(n):\n result = 1\n ...,def find_demlo_number(n):\n return math.pro...,def find_demlo_number(n):\n result = 1\n ...,28.984074,-37.834845,-70.106065,25.626914,-42.058151,-71.300691
511,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,def count_elements_until_tuple(my_list):\n ...,37.437273,-38.654938,-63.018478,33.775540,-40.445251,-65.945331
149,def decimal_to_binary(n):\n return bin(n)[2:],"def decimal_to_binary(n):\n binary_str = """"...",def decimal_to_binary(n):\n binary_string =...,def decimal_to_binary(n):\n binary = ''\n ...,def decimal_to_binary(n):\n binary_str = ''...,def decimal_to_binary(n):\n binary_string =...,def decimal_to_binary(n):\n return format(n...,def decimal_to_binary(n):\n return bin(n).r...,def decimal_to_binary(n):\n if n == 0:\n ...,11.896458,-23.381871,-34.726561,34.173141,-26.324015,-46.342534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n n = len(ma...,def find_max_avg_path(matrix):\n dp = [[0] ...,def find_max_avg_path(matrix):\n n = len(ma...,130.438136,-35.015840,-99.392036,162.611514,-37.374619,-110.195254
801,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n positive_co...,def find_positive_ratio(arr):\n return sum(...,def find_positive_ratio(arr):\n return sum(...,def find_positive_ratio(arr):\n positive_co...,40.542724,-29.502470,-62.003786,31.214260,-30.276142,-60.052932
564,def count_set_bits(n):\n count = 0\n whi...,def count_set_bits(n):\n count = 0\n whi...,def count_set_bits(n):\n count = 0\n whi...,def count_set_bits(n):\n count = 0\n whi...,def count_set_bits(n):\n count = 0\n whi...,def count_set_bits(n):\n count = 0\n whi...,def count_s

In [15]:
hum_score

[-12.795919482756062,
 -39.09079210971326,
 -6.856628836824584,
 -29.15920871592877,
 -1.160190123684032,
 -14.103850814112796,
 -26.21829378743962,
 -17.73360359185122,
 -21.549867033324176,
 -16.257334942310266,
 -11.476469108326242,
 -19.43589778611603,
 -48.26072526173759,
 -14.536602596760005,
 -5.087981189202338,
 -1.7692774954801394,
 -4.521651158611334,
 -35.360664044531134,
 -27.244134200804794,
 -13.737392491276495,
 -11.248722848018971,
 -18.561898276036118,
 -17.915443179843265,
 -2.76176891170595,
 -11.589406157932132,
 -9.856003730537203,
 -13.29437129027444,
 -11.846360318600176,
 -7.208948214376164,
 -20.099311037532786,
 -30.424415698443383,
 -9.668436238718591,
 -15.912913727470652,
 -12.562772212118816,
 -1.723599417122749,
 -10.376678864428797,
 0.4269300325190386,
 -21.724843909941697,
 -30.812432571674535,
 5.96534098598768,
 -9.226327334526843,
 -15.241171425258287,
 2.3735857346863583,
 -11.828946576202455,
 -30.754356632383207,
 -17.36972460538329,
 -24.1783655